In [1]:
import os
os.chdir('../')
%pwd

'h:\\Personal Projects\\Carbon-Footprint-estimation'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str  # e.g., "XGBoost"
    n_estimators: int
    max_depth: int
    learning_rate: float
    subsample: float
    colsample_bytree: float
    reg_alpha: float
    reg_lambda: float
    min_child_weight: int
    random_state: int
    objective: str
    target_column: str


In [3]:
from CarbonFootprint.constants import *
from CarbonFootprint.utils.common import read_yaml, create_directories

In [7]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XgBoost
        target_column_name = list(self.schema.TARGET_COLUMN.keys())[0]  # Gets "CarbonEmission"

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            learning_rate=params.learning_rate,
            subsample=params.subsample,
            colsample_bytree=params.colsample_bytree,
            reg_alpha=params.reg_alpha,
            reg_lambda=params.reg_lambda,
            min_child_weight=params.min_child_weight,
            random_state=params.random_state,
            objective=params.objective,
            target_column=target_column_name
        )

        return model_trainer_config


In [8]:
import pandas as pd
import os
from CarbonFootprint import logger
from xgboost import XGBRegressor
import joblib

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Split features and target
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

       
        model = XGBRegressor(
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            learning_rate=self.config.learning_rate,
            subsample=self.config.subsample,
            colsample_bytree=self.config.colsample_bytree,
            reg_alpha=self.config.reg_alpha,
            reg_lambda=self.config.reg_lambda,
            min_child_weight=self.config.min_child_weight,
            random_state=self.config.random_state,
            objective=self.config.objective,
        )

        
        model.fit(train_x, train_y)

        # Save the model
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-05-13 21:09:24,843: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-13 21:09:24,845: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-13 21:09:24,846: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-13 21:09:24,850: INFO: common: created directory at: artifacts]
[2025-05-13 21:09:24,853: INFO: common: created directory at: artifacts/model_trainer]
